<a href="https://colab.research.google.com/github/Codekster/RAG-practice/blob/main/RAGTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_community langchainhub chromadb langchain langchain-openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:

In [ ]:
!pip install langchain_google_genai

In [36]:
from google.colab import userdata
import os

# Assuming your Gemini API key is stored as 'GOOGLE_API_KEY' in Colab secrets
gemini_api_key = userdata.get('Googlekey')
os.environ['GOOGLE_API_KEY'] = gemini_api_key

print("Gemini API key has been set as an environment variable.")

Gemini API key has been set as an environment variable.


In [37]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = ["https://en.wikipedia.org/wiki/World_War_II"]

loader = WebBaseLoader(web_paths=page_url)
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

print(f"Fetched {len(docs)} documents")

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.86it/s]


Fetched 1 documents


In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=3000,
    chunk_overlap=300,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(docs)
print(texts[0])
print(texts[1])


page_content='World War II - Wikipedia



































Jump to content







Main menu





Main menu
move to sidebar
hide



		Navigation
	


Main pageContentsCurrent eventsRandom articleAbout WikipediaContact us





		Contribute
	


HelpLearn to editCommunity portalRecent changesUpload fileSpecial pages



















Search











Search






















Appearance
















Donate

Create account

Log in








Personal tools





Donate Create account Log in





		Pages for logged out editors learn more



ContributionsTalk




























Contents
move to sidebar
hide




(Top)





1
Start and end dates








2
Background




Toggle Background subsection





2.1
Aftermath of World War I








2.2
Germany and Italy








2.3
European treaties








2.4
Asia










3
Pre-war events




Toggle Pre-war events subsection





3.1
Italian invasion of Ethiopia (1935)








3.2
Spanish Civil War (1936–1939)








3.3


In [39]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
import os

# Ensure GOOGLE_API_KEY is set from the previous cell
if 'GOOGLE_API_KEY' not in os.environ:
    print("GOOGLE_API_KEY not set. Please run the cell to set the environment variable.")
else:
    vectorstore = Chroma.from_documents(documents=texts, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
    print("Embeddings created and stored in Chroma.")

Embeddings created and stored in Chroma.


In [41]:
retriever=vectorstore.as_retriever()

In [42]:
from langchain import hub
prompt=hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [43]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# Ensure GOOGLE_API_KEY is set
if 'GOOGLE_API_KEY' not in os.environ:
    print("GOOGLE_API_KEY not set. Please run the cell to set the environment variable.")
else:
    llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", temperature=0.5)
    print("Gemini LLM instance created.")

Gemini LLM instance created.


In [44]:
def promptJoin(prompt):
  return "\n".join(doc.page_content for doc in prompt )

In [32]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
rag_chain=({"context":retriever | promptJoin ,"question":RunnablePassthrough()}
| prompt
| llm
| StrOutputParser()
)

In [48]:
rag_chain.invoke("who faught best in the war")

"I don't know the answer. The provided context contains information about Wikipedia's policies, licensing, and general links related to World War II, but it does not discuss who fought best in the war."